In [97]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass
import json

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *
from kpi.match import *
from kpi.request import *
from kpi.where import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.kpi import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/sam11/Git/nestor


# Database

In [98]:
user = "neo4j"
password="GREYSTONE!!"

In [99]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)
database = DatabaseNeo4J("bolt://127.0.0.1:7687", user, password, databaseSchema_dict)

yaml file open


# Query

In [108]:
kpiMachine = Kpi(match = '1-machine', where = 'machine.name = "h1"')
kpiIssue = Kpi(match = '2-issue', where = 'issue.cost<10')
kpiIssue1 = Kpi(match = '3-issue', result = 'issue.description_of_problem')
kpiTechnician =  Kpi(match = '4-technician', where = 'technician.name CONTAINS "bob"')


In [109]:
kpi =  kpiTechnician + (kpiIssue > kpiMachine)

---------------------
['1-machine-->2-issue']
['1-machine-->2-issue']


In [110]:
print(kpi)

4-technician / 1-machine-->2-issue
technician.name CONTAINS "bob" AND machine.name = "h1" AND issue.cost<10




In [92]:
t = kpi > (kpiTechnician > kpiIssue1) +

---------------------
['3-issue-->4-technician']
['3-issue', '3-issue-->4-technician']
---------------------
['3-issue-->4-technician-->2-issue']
['3-issue', '3-issue-->4-technician', '3-issue-->4-technician-->2-issue']


In [70]:
t.match

['4-technician', '4-technician-->3-issue']

In [31]:
print(kpi.cypherQuery())

MATCH (machine:MACHINE), (machine:MACHINE)-->(technician:TECHNICIAN:HUMAN)
WHERE machine.name = "h1" AND technician.name CONTAINS "bob"
RETURN 


-------

## object

In [ ]:
item = TagItemKpi(operator="=", keyword="valve", databaseInfo=database.schema)
machine = MachineKpi(operator="<>", name = "h1", databaseInfo=database.schema)
technician = TechnicianKpi(operator="-", name="j", databaseInfo=database.schema )

## result

In [ ]:
problem = TagProblemKpi(result="keyword",databaseInfo=database.schema)

## Operation

In [ ]:
result = item > technician +  problem

## result 

In [ ]:
print(result.cypher_createQuery())

In [ ]:
res = database.runQuery(result.cypher_createQuery())[1]

In [ ]:
dataframe = resultToObservationDataframe(res)

In [ ]:
dataframe